In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from segment_anything import sam_model_registry, SamPredictor
from PIL import Image
import os
import yaml
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# YAML 파일 로드
with open('C:/Users/AI-LHJ/Desktop/Wire2/data.yaml', 'r') as file:
    config = yaml.safe_load(file)

# SAM 모델 로드
sam_checkpoint = "sam_b.pt"
model_type = "vit_b"
device = "cuda" if torch.cuda.is_available() else "cpu"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)

# 커스텀 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.images = [f for f in os.listdir(img_dir) if f.endswith('.jpg') or f.endswith('.png')]
        self.class_to_idx = {name: idx for idx, name in enumerate(config['names'])}

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        # numpy 배열로 변환
        image_np = np.array(image.permute(1, 2, 0))

        # SAM을 사용하여 세그멘테이션 수행
        predictor.set_image(image_np)
        masks, _, _ = predictor.predict(multimask_output=True)

        # 가장 큰 세그먼트를 선택 (예시)
        largest_mask = masks[np.argmax([m.sum() for m in masks])]

        # 마스크의 평균값을 특징으로 사용
        features = torch.tensor(largest_mask.mean()).unsqueeze(0).float()

        # 클래스 레이블 (실제 구현에서는 이미지에 해당하는 정확한 레이블을 제공해야 함)
        label = torch.tensor(self.class_to_idx['crossarm']).long()  # 예시로 'crossarm' 클래스 사용

        return features, label

# 데이터 로더 설정
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

train_dataset = CustomDataset(config['train'], transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 간단한 분류기 모델
class SimpleClassifier(nn.Module):
    def __init__(self, num_classes):
        super(SimpleClassifier, self).__init__()
        self.fc = nn.Linear(1, num_classes)

    def forward(self, x):
        return self.fc(x)

# 모델, 손실 함수, 옵티마이저 설정
model = SimpleClassifier(len(config['names']))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 성능 평가 함수
def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, precision, recall, f1

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    all_labels = []
    all_predictions = []
    
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
    
    # 에폭당 평균 손실 계산
    epoch_loss = running_loss / len(train_loader)
    
    # 성능 지표 계산
    accuracy, precision, recall, f1 = compute_metrics(all_labels, all_predictions)
    
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    print(f'Loss: {epoch_loss:.4f}')
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print('-' * 50)

print("Training finished!")

# 모델 저장
torch.save(model.state_dict(), 'sam_classifier.pth')

# 전체 데이터셋에 대한 최종 평가
model.eval()
all_labels = []
all_predictions = []

with torch.no_grad():
    for features, labels in train_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

final_accuracy, final_precision, final_recall, final_f1 = compute_metrics(all_labels, all_predictions)

print("Final Evaluation Results:")
print(f'Accuracy: {final_accuracy:.4f}')
print(f'Precision: {final_precision:.4f}')
print(f'Recall: {final_recall:.4f}')
print(f'F1 Score: {final_f1:.4f}')

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from segment_anything import sam_model_registry, SamPredictor

# 커스텀 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, image_paths, mask_paths):
        self.image_paths = image_paths
        self.mask_paths = mask_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # 이미지와 마스크 로드 및 전처리
        # 실제 구현에서는 이미지 로딩 및 전처리 로직을 추가해야 합니다
        return image, mask

# 데이터 로더 설정
train_dataset = CustomDataset(train_image_paths, train_mask_paths)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# SAM 모델 로드
model = sam_model_registry["default"](checkpoint="path/to/sam_vit_h_4b8939.pth")

# 미세 조정을 위해 일부 레이어만 학습 가능하도록 설정
for param in model.image_encoder.parameters():
    param.requires_grad = False

# 마스크 디코더의 마지막 몇 개 레이어만 학습 가능하도록 설정
for param in model.mask_decoder.transformer.layers[-2:].parameters():
    param.requires_grad = True

# 손실 함수와 옵티마이저 정의
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

# 학습 루프
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)

        # 전방 전파
        outputs = model(images)
        loss = criterion(outputs, masks)

        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# 학습된 모델 저장
torch.save(model.state_dict(), "fine_tuned_sam_model.pth")

# 미세 조정된 모델을 사용한 예측
predictor = SamPredictor(model)
predictor.set_image(test_image)
masks, _, _ = predictor.predict(point_coords=None, point_labels=None, box=None)